In [3]:
from qiskit import *
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')
provider.backends()

from qiskit.providers.ibmq import least_busy

small_devices = provider.backends(filters=lambda x: x.configuration().n_qubits == 5
                                   and not x.configuration().simulator)
least_busy(small_devices)

/usr/local/lib/python3.7/dist-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


<IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>

In [4]:
simulator = BasicAer.get_backend('qasm_simulator')
backend = provider.get_backend('ibmq_valencia')

In [5]:
# supplementary
def relerror(a, b):
    return abs(a - b) / max(abs(a), abs(b))

In [4]:
# problem 1
from math import *
circuit = QuantumCircuit(1, 1)
phi = 0.9
theta = 0.5
circuit.u3(pi - 2 * theta, phi, 0, 0)
circuit.h(0)
circuit.measure([0], [0])
circuit.draw()

┌──────────────────┐┌───┐┌─┐
q_0: ┤ U3(2.1416,0.9,0) ├┤ H ├┤M├
     └──────────────────┘└───┘└╥┘
c: 1/══════════════════════════╩═
                               0

In [10]:
shots = 2048
job = execute(circuit, simulator, shots=shots)
result = job.result()
counts = result.get_counts(circuit)
print(counts)
for k in counts.keys():
    counts[k] /= shots
c0 = (counts['0'] - counts['1'])
cosphi = c0 / cos(theta) / sin(theta) / 2
print("prepared = {}, measured = {}, relative error = {}".format(cos(phi), cosphi, relerror(cosphi, cos(phi))))

{'1': 514, '0': 1534}
prepared = 0.6216099682706644, measured = 0.5918764686980877, relative error = 0.047833048197885315


In [6]:
#problem 2
from math import *
circuit = QuantumCircuit(4, 4)
phi = pi / 3
theta = pi / 3
# prepare 2 equal states
circuit.h(0)
circuit.u1(2 * theta, 0)
circuit.h(0)
circuit.s(0)
circuit.x(0)
circuit.u1(phi, 0)
circuit.h(1)
circuit.u1(2 * theta, 1)
circuit.h(1)
circuit.s(1)
circuit.x(1)
circuit.u1(phi, 1)
# same with u3
circuit.u3(pi - 2 * theta, phi, 0, 2)
circuit.u3(pi - 2 * theta, phi, 0, 3)
# use qbits 0, 2 to check theta, use qbits 1, 3 to check phi
# prepare qbits 1, 3 for phi measurements
circuit.h(1)
circuit.h(3)
circuit.measure([0, 1, 2, 3], [0, 1, 2, 3])
circuit.draw()

┌───┐       ┌───────────┐┌───┐┌───┐┌───┐┌──────────┐     ┌─┐   
q_0: ───────┤ H ├───────┤ U1(2pi/3) ├┤ H ├┤ S ├┤ X ├┤ U1(pi/3) ├─────┤M├───
            ├───┤       ├───────────┤├───┤├───┤├───┤├──────────┤┌───┐└╥┘┌─┐
q_1: ───────┤ H ├───────┤ U1(2pi/3) ├┤ H ├┤ S ├┤ X ├┤ U1(pi/3) ├┤ H ├─╫─┤M├
     ┌──────┴───┴──────┐└───────────┘└┬─┬┘└───┘└───┘└──────────┘└───┘ ║ └╥┘
q_2: ┤ U3(pi/3,pi/3,0) ├──────────────┤M├─────────────────────────────╫──╫─
     ├─────────────────┤    ┌───┐     └╥┘  ┌─┐                        ║  ║ 
q_3: ┤ U3(pi/3,pi/3,0) ├────┤ H ├──────╫───┤M├────────────────────────╫──╫─
     └─────────────────┘    └───┘      ║   └╥┘                        ║  ║ 
c: 4/══════════════════════════════════╩════╩═════════════════════════╩══╩═
                                       2    3                         0  1

In [7]:
from math import *
shots = 65536 # 65536 for simulation, 1024 for real system
job = execute(circuit, simulator, shots=shots)
result = job.result()
counts = result.get_counts(circuit)
print(counts)
for k in counts.keys():
    counts[k] /= shots
# separate qbit measurements
def gettrace(counts, n, mask):
    p = 0
    for i in range(1 << n):
        state = ''
        for j in range(n):
            state += str((i >> j) & 1)
        corresponds = True
        for j in range(n):
            if mask[j] != '?' and mask[j] != state[j]:
                corresponds = False
        if not corresponds:
            continue
        p += counts[state]
    return p
# p[i][j] is the probability of qbit i being in state j
p = [(gettrace(counts, 4, '?' * (3 - i) + '0' + '?' * i),
      gettrace(counts, 4, '?' * (3 - i) + '1' + '?' * i)) for i in range(4)]

sintheta = sqrt(p[0][0])
costheta = sqrt(p[0][1])
c0 = (p[1][0] - p[1][1])
cosphi = c0 / cos(theta) / sin(theta) / 2
print("manual: max error = {}".format(max(relerror(sintheta, sin(theta)), relerror(costheta, cos(theta)), relerror(cosphi, cos(phi)))))
sintheta = sqrt(p[2][0])
costheta = sqrt(p[2][1])
c0 = (p[3][0] - p[3][1])
cosphi = c0 / cos(theta) / sin(theta) / 2
print("using u3: max error = {}".format(max(relerror(sintheta, sin(theta)), relerror(costheta, cos(theta)), relerror(cosphi, cos(phi)))))

{'1100': 2485, '0001': 6288, '0000': 19020, '0010': 7564, '1000': 7429, '1101': 864, '1001': 2464, '1110': 1004, '0110': 2426, '0011': 2516, '0100': 6359, '1010': 2958, '1011': 949, '0101': 2053, '0111': 823, '1111': 334}
manual: max error = 0.002842173741413378
using u3: max error = 0.006444911728573029
